Çok Değişkenli Veri Analizi
Pivot Tablosu
Pivot tablo , veritabanı, iş zekası programı ya da veri tablosu gibi daha geniş bir tabloyu özetleyen istatistik tabosudur. Bu özet, pivot tabloyu önemli bir şekilde gruplayan toplamları, ortalamaları veya diğer istatistikleri içerebilir.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

#python'da uyarıları kapatalım
import warnings
warnings.filterwarnings("ignore")

#veriyi içeri aktaracağız
veri=pd.read_csv("olimpiyatlar_temizlenmis.csv")

In [3]:
veri_pivot=veri.pivot_table(index="madalya",columns='cinsiyet',
                values=["boy","kilo","yas"],
                aggfunc={"boy":np.mean,'kilo':np.mean,'yas':[min,max,np.std]})
veri_pivot.head()

boy                   kilo              yas              \
                mean                   mean              max         min   
cinsiyet           F           M          F          M     F     M     F   
madalya                                                                    
Bronze    170.003227  180.045806  62.757125  77.841504  69.0  72.0  12.0   
Gold      170.448727  180.318906  63.199349  78.186505  63.0  64.0  13.0   
Silver    170.233783  180.053626  62.866892  77.960887  55.0  73.0  11.0   

                                    
                     std            
cinsiyet     M         F         M  
madalya                             
Bronze    10.0  5.329229  5.870340  
Gold      13.0  5.219615  5.987807  
Silver    13.0  5.253111  6.098221